In [1]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_119654/65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [2]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[4946714006, 1794905007, 9712783009, 739546401..."
1,Name,object,14,0.00,779583,"[KELLAR, HAYS & LYNCH, P.C., CREATIVE SCHOOLHO..."
2,City,object,30,0.00,32581,"[PITTSBURGH, DURHAM, MILWAUKEE, SAN ANTONIO, B..."
3,State,object,14,0.00,51,"[MT, TX, OK, CA]"
4,Zip,int64,0,0.00,33611,"[85022, 90703, 55344, 49428, 87501]"
5,Bank,object,1559,0.17,5802,"[TEXAS FIRST BANK, JPMORGAN CHASE BANK NATL AS..."
6,BankState,object,1566,0.17,56,"[MN, NC, UT, CA]"
7,NAICS,int64,0,0.00,1312,"[811122, 484110, 236115, 446191, 812112]"
8,ApprovalDate,object,0,0.00,9859,"[8-Nov-90, 10-Nov-86, 7-Nov-03, 22-Jan-09, 11-..."
9,ApprovalFY,object,0,0.00,70,"[2001, 1991, 1997, 2003]"


In [3]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [4]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [5]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [6]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [7]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [8]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [9]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [10]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [11]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [12]:
df.shape

(675026, 13)

In [13]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[44-45, 31-33, 48-49, 62]"
1,Term,int64,0,0.0,377,"[120, 84, 21, 83, 22]"
2,NoEmp,int64,0,0.0,499,"[4, 16, 2, 1]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[9, 0]"
5,RetainedJob,int64,0,0.0,323,"[0, 15, 37, 5]"
6,UrbanRural,int64,0,0.0,3,"[1, 2, 0]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[50000, 75000, 140000, 80800, 25000]"


In [14]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [15]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_119654/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [16]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [17]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [18]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [19]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [20]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [21]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [22]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [23]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [24]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [25]:
df.shape

(460816, 13)

In [26]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 31-33, 23, 44-45]"
1,Term,int64,0,0.0,373,"[264, 84, 240]"
2,NoEmp,int64,0,0.0,432,"[1, 4, 30, 3]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,192,[0]
5,RetainedJob,int64,0,0.0,276,"[2, 54, 0]"
6,UrbanRural,int64,0,0.0,3,"[1, 0, 2]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,12824,"[26000, 25000, 250000, 50000]"


In [27]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [28]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_119654/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [29]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [30]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [31]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [32]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [33]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [34]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [35]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [36]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [37]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [38]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [39]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [40]:
df.shape

(675026, 13)

In [41]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[42, 44-45, 48-49, 23]"
1,Term,int64,0,0.0,377,"[75, 14, 84, 175, 36]"
2,NoEmp,int64,0,0.0,499,"[2, 1, 10]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 5, 10]"
5,RetainedJob,int64,0,0.0,323,"[0, 1, 6, 11]"
6,UrbanRural,int64,0,0.0,3,[1]
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[25000, 75000, 50000, 565000, 45000]"


In [42]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [43]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_119654/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [44]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[7719543000, 5359983010, 1856394009, 750507400..."
1,Name,object,14,0.00,779583,"[THE PAPER PEDDLER, REBECCA BIORATO INC, Etien..."
2,City,object,30,0.00,32581,"[EUGENE, CLAYMONT, HOUSTON, VALLEY CENTER, OAK..."
3,State,object,14,0.00,51,"[UT, FL, LA, CA]"
4,Zip,int64,0,0.00,33611,"[79606, 6812, 55904, 6488, 80111]"
5,Bank,object,1559,0.17,5802,"[SARATOGA NATL BK & TR CO, OLD FLORIDA BANK, F..."
6,BankState,object,1566,0.17,56,"[RI, ME, CA, DC, DE]"
7,NAICS,int64,0,0.00,1312,"[0, 327310, 713910, 313222, 621111]"
8,ApprovalDate,object,0,0.00,9859,"[22-Apr-99, 24-Dec-98, 29-Jul-96, 29-Oct-99, 1..."
9,ApprovalFY,object,0,0.00,70,"[2001, 1995, 2004, 1991]"


In [45]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [46]:
is_valide = {1.0, 2.0}
df['NewExist'] = df['NewExist'].isin(is_valide)

In [47]:
is_valide = {'N','Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide)

In [48]:
is_valide = {'N', 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide)

In [49]:
is_valide = {'P I F', 'CHGOFF'}
df['MIS_Status'] = df['MIS_Status'].isin(is_valide)

In [50]:
df = df.dropna(subset='NAICS')
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [51]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [52]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:])
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [53]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [54]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [55]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [56]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [57]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [58]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[DENVER, WEBSTER, BROOKLYN, Acton, CLACKAMAS]"
1,State,object,0,0.0,51,"[MO, IL, NJ, HI, FL]"
2,Bank,object,0,0.0,5221,"[U.S. BANK NATIONAL ASSOCIATION, FULTON BANK N..."
3,BankState,object,0,0.0,55,"[RI, CT, NY, IL]"
4,NAICS,object,0,0.0,20,"[31-33, 44-45]"
5,ApprovalDate,object,0,0.0,7367,"[10-Aug-95, 27-Jun-07, 3-Sep-04, 21-Jan-98, 23..."
6,ApprovalFY,object,0,0.0,29,"[2011, 2008, 2000, 2006, 2002]"
7,Term,int64,0,0.0,378,"[84, 60, 24, 48]"
8,NoEmp,int64,0,0.0,502,"[1, 9, 10, 7, 2]"
9,NewExist,bool,0,0.0,2,[True]


In [59]:
df.shape

(696583, 19)

In [60]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
